# CS:GO Cheater Probability Calculator
# 4. Testing the model with out-of-sample accounts

In [74]:
from sklearn.ensemble import RandomForestClassifier
from joblib import load
import requests
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

rf_classifier = load('cheater_proba_rf_classifier.joblib')

steam_id = input("Please, enter Steam64 ID:")
api_key = 'X' # Replace with your own API key if you are going to use this code
app_id = "730"

def check_stats():
    # 1. Get Stats
    stats = []
    url = f'https://api.steampowered.com/ISteamUserStats/GetUserStatsForGame/v2/?appid={app_id}&key={api_key}&steamid={steam_id}'
    response = requests.get(url)
    data = response.json()
    if 'playerstats' in data:
        playerstats = data['playerstats']
        if 'stats' in playerstats:
            stats_data = playerstats['stats']
            stats_dict = {} #{'steam_id': steam_id}
            for stat in stats_data:
                stats_dict[stat['name']] = stat['value']
            stats.append(stats_dict)
    else:
        print("This account does not allow stat tracking")
        return

    # 2. Create features
    df = pd.DataFrame(stats)
    df['win_%'] = df['total_matches_won'] / df['total_matches_played']
    df["kd"] = df['total_kills'] / df['total_deaths']
    df["hs_%"] = df['total_kills_headshot'] / df['total_kills']
    df["accuracy"] = df['total_shots_hit'] / df['total_shots_fired']
    df["mvp_%"] = df["total_mvps"] / df['total_rounds_played']
    df["dmg_round"] = df["total_damage_done"] / df['total_rounds_played']
    df["contribution_round"] = df["total_contribution_score"] / df['total_rounds_played']
    matches_played = df["total_matches_played"][0]
    columns = ['win_%',"hs_%","mvp_%","kd","dmg_round","accuracy","contribution_round"]
    df = df[columns].copy()

    # 3. Run Model
    n_df = np.log1p(df)
    y_pred = rf_classifier.predict(n_df)
    class_probabilities = rf_classifier.predict_proba(n_df)
    probability = int(class_probabilities[:, 1]*100)

    # 4. Return results
    df["matches_played"] = matches_played 
    print(f"\nPlayer Stats:")
    display(df.style.hide_index())
    print(f"\nProbability of Cheating: {probability}%")
    return

check_stats()
# Testing with my own account

Please, enter Steam64 ID: 76561198047193521



Player Stats:


win_%,hs_%,mvp_%,kd,dmg_round,accuracy,contribution_round,matches_played
0.460227,0.354415,0.124028,0.894301,126.377862,0.220786,2.102134,2640



Probability of Cheating: 22%


In [58]:
# Vanity URL to Steam64 ID converter
vanity_url = input("Please enter vanity name:") # Only the XXX in https://steamcommunity.com/id/XXX/
url = f"http://api.steampowered.com/ISteamUser/ResolveVanityURL/v0001/?key={api_key}&vanityurl={vanity_url}"
response = requests.get(url)
data = response.json()
data["response"]["steamid"]

'76561198047193521'